In [ ]:
from utils import get_context, get_response

# **Intent Recognition using Chain-of-Thought Reasoning and Inflection AI API**

This notebook implements **intent recognition** with **Chain-of-Thought (CoT) reasoning** to enhance service classification and decision-making. The goal is to optimize intent detection and structured output generation for downstream automation.

## **Overview**

- **Chain-of-Thought reasoning** improves intent classification accuracy.
- Uses **structured prompting** to ensure consistent output.
- Implements a multi-step approach:
  - **Extracts intent** from user provided emails.
  - **Generates reasoning traces** before classification.
  - **Outputs results in structured format** for seamless automation.
- Leverages the Inflection AI model **inflection_3_productivity model**, optimized for task-specific classification.

This approach ensures **interpretable, structured, and automation-friendly intent classification**, improving routing efficiency.

In [ ]:
legacy_api = False # True if using the old API, False if using the new OpenAI Compatible API

In [ ]:
system_instruction_prompt = """
You are an AI assistant designed to classify user messages by intent.

# Your Purpose
1. Identify the intent of the human's message without attempting to solve their request.
2. Choose ONLY from the predefined valid intents listed below.
3. DO NOT create new intents or modify existing ones.
4. Always review all valid intents before making a selection.

# Definitions
- availability_request: The email requests availability for a meeting.
Examples:
- "Do you have time tomorrow for an hour?"
- "What time slots do you have available next week in Pacific?"
- "Are you available for a meeting?"
- "Can you meet at 10:00 am?"
- "What time works for you?"
- "When can we meet?"
- "Can we schedule a meeting?"
- "Are you free to meet?"
- "Can we meet tomorrow?

- bug_report: The email reports an issue or bug.
Examples:
- "I found a bug in the system."
- "The system is not working as expected."
- "I encountered an issue."
- "There is a problem with the system."
- "I am facing an error."
- "The system is not functioning properly."

- no_reply_needed_or_spam: The email does not require a reply or is classified as spam.
Examples:
- "Declined invitation."
- "Accepted invitation."
- "Invitation sent."

- reply_as_normal_with_pi: The email does not fit into any of the other predefined categories.

# Valid Intents
- availability_request
- bug_report
- no_reply_needed_or_spam
- reply_as_normal_with_pi


# Output Format - XML
You will respond using XML tags for each extracted piece of information. You don't need to provide explanation or any other information, just return the extracted parts within the appropriate XML tags.

# Format of the Output
<parts>
    <reasoning>The reasoning behind selecting this intent</reasoning>
    <intent_recognized>intent_selected</intent_recognized>
</parts>
"""

## Test Scenario: Intent Recognition of Short Emails

In [ ]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
async def test_classify_email_intent():
    print("Starting test: test_classify_email_intent")
    """
    Scenario: The user provides email bodies that contain different intents.
    Expected Outcome: The system should be able to recognize the intent of the email body. It also provides reasoning for the intent selected.
    """
    print("+*"*20)
    email_body = "Good day! I stumbled upon a charming little bug in one of your functions, please fix."

    context = get_context(system_instruction_prompt, email_body, user_input_label="Email body", legacy_api=legacy_api)
    result = await get_response(context, ["reasoning", "intent_recognized"], legacy_api=legacy_api)
    assert result["intent_recognized"] == "bug_report"
    print(f"{color.BOLD} Intent recognized {color.END} {result["intent_recognized"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    email_body = "Do you have time to meet today to strategize our next masterstroke, or shall we let the universe think it's gotten the upper hand?"
    context = get_context(system_instruction_prompt, email_body, user_input_label="Email body", legacy_api=legacy_api)
    result = await get_response(context, ["reasoning", "intent_recognized"], legacy_api=legacy_api)
    assert result["intent_recognized"] == "availability_request"
    print(f"{color.BOLD} Intent recognized: {color.END} {result["intent_recognized"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    email_body = "The invitation to the meeting was regrettably declined."
    context = get_context(system_instruction_prompt, email_body, user_input_label="Email body", legacy_api=legacy_api)
    result = await get_response(context, ["reasoning", "intent_recognized"], legacy_api=legacy_api)
    assert result["intent_recognized"] == "no_reply_needed_or_spam"
    print(f"{color.BOLD} Intent recognized: {color.END} {result["intent_recognized"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    email_body = "What's the capital of France? Hint: It's not 'F'."
    context = get_context(system_instruction_prompt, email_body, user_input_label="Email body", legacy_api=legacy_api)
    result = await get_response(context, ["reasoning", "intent_recognized"], legacy_api=legacy_api)
    assert result["intent_recognized"] == "reply_as_normal_with_pi"
    print(f"{color.BOLD} Intent recognized: {color.END} {result["intent_recognized"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    print("All tests passed successfully! 🙌")

# Run the test
await test_classify_email_intent()